## 5. H2O AutoML and Hyperparameter Search

We run AutoML after establishing baselines. AutoML explores multiple model families and hyperparameters to find strong candidates.



## 1. Setup

We load core libraries and enable the local H2O environment.

Logs will be written to ./h2o_logs and H2O will run in verbose mode.


In [1]:
import os
import sys
import numpy as np
import pandas as pd

# Ensure h2o is available from /tmp/pydeps
sys.path.insert(0, '/tmp/pydeps')
import h2o
from h2o.automl import H2OAutoML

pd.set_option('display.max_columns', 200)
np.random.seed(42)





## 2. Data loading

We load the OULAD CSV files and build the same final dataset used in the main notebook.


In [2]:
# CHANGE THIS to your local folder containing the 7 OULAD CSV files
DEFAULT_DATA_DIR = os.path.join('.', 'data', 'oulad')

required_files = [
    'assessments.csv','courses.csv','studentAssessment.csv','studentInfo.csv',
    'studentRegistration.csv','studentVle.csv','vle.csv'
]

# Resolve DATA_DIR from common locations
candidate_dirs = [DEFAULT_DATA_DIR, '.']
resolved = None
for d in candidate_dirs:
    if all(os.path.exists(os.path.join(d, f)) for f in required_files):
        resolved = d
        break

if resolved is None:
    missing = [f for f in required_files if not os.path.exists(os.path.join(DEFAULT_DATA_DIR, f))]
    print('Missing files in DEFAULT_DATA_DIR:', missing)
    print('DEFAULT_DATA_DIR currently set to:', os.path.abspath(DEFAULT_DATA_DIR))
    print('Also checked:', os.path.abspath('.'))
else:
    DATA_DIR = resolved
    print('Using DATA_DIR:', os.path.abspath(DATA_DIR))


Using DATA_DIR: d:\Project\DBM_FINAL


In [3]:
def read_csv(name, usecols=None, dtype=None):
    path = os.path.join(DATA_DIR, name)
    return pd.read_csv(path, usecols=usecols, dtype=dtype, low_memory=False)

assessments = read_csv('assessments.csv')
courses = read_csv('courses.csv')
student_info = read_csv('studentInfo.csv')
student_reg = read_csv('studentRegistration.csv')
student_assess = read_csv('studentAssessment.csv')
vle = read_csv('vle.csv')

student_vle = read_csv(
    'studentVle.csv',
    usecols=['code_module','code_presentation','id_student','id_site','date','sum_click'],
    dtype={
        'code_module':'category','code_presentation':'category',
        'id_student':'int32','id_site':'int32',
        'date':'int16','sum_click':'int32'
    }
)


## 3. Cleaning and feature engineering

We apply the same cleaning and feature engineering used in the main notebook, then drop leakage features and remove zero-activity rows.


In [4]:
# --- assessments: drop known invalid IDs
invalid_ids = {40087, 40088}
assessments = assessments[~assessments['id_assessment'].isin(invalid_ids)].copy()
assessments['weight'] = pd.to_numeric(assessments['weight'], errors='coerce')
assessments['date'] = pd.to_numeric(assessments['date'], errors='coerce')

# --- studentAssessment: clean score/date
student_assess['score'] = student_assess['score'].replace(['?', '', ' '], np.nan)
student_assess['score'] = pd.to_numeric(student_assess['score'], errors='coerce')
student_assess['date_submitted'] = pd.to_numeric(student_assess['date_submitted'], errors='coerce')
student_assess['is_banked'] = pd.to_numeric(student_assess['is_banked'], errors='coerce').fillna(0).astype(int)
student_assess = student_assess.dropna(subset=['score']).copy()

# --- studentInfo cleanup
student_info['imd_band'] = student_info['imd_band'].replace('?', np.nan)
student_info['age_band'] = student_info['age_band'].replace({'55<=': '>=55'})

# --- studentVle: drop duplicates
student_vle = student_vle.drop_duplicates().copy()


In [5]:
# --- target
student_info['final_result_upd'] = student_info['final_result'].replace({
    'Distinction': 'Pass',
    'Withdrawn': 'Fail'
})
student_info = student_info[student_info['final_result_upd'].isin(['Pass','Fail'])].copy()
student_info['final_result_upd_numeric'] = (student_info['final_result_upd'] == 'Pass').astype(int)

# --- VLE aggregates
vle_agg = (
    student_vle
    .groupby(['id_student','code_module','code_presentation'], as_index=False, observed=True)
    .agg(
        total_sum_click=('sum_click','sum'),
        active_days=('date','nunique'),
        distinct_sites=('id_site','nunique')
    )
)

early = student_vle[student_vle['date'] <= 14]
early_agg = (
    early.groupby(['id_student','code_module','code_presentation'], as_index=False, observed=True)
         .agg(early_sum_click=('sum_click','sum'), early_active_days=('date','nunique'))
)

vle_features = vle_agg.merge(early_agg, on=['id_student','code_module','code_presentation'], how='left')
vle_features[['early_sum_click','early_active_days']] = vle_features[['early_sum_click','early_active_days']].fillna(0)

vle_features['early_click_ratio'] = vle_features['early_sum_click'] / vle_features['total_sum_click'].replace(0, np.nan)
vle_features['early_click_ratio'] = vle_features['early_click_ratio'].fillna(0)

vle_features['clicks_per_active_day'] = vle_features['total_sum_click'] / vle_features['active_days'].replace(0, np.nan)
vle_features['clicks_per_active_day'] = vle_features['clicks_per_active_day'].fillna(0)

vle_features['early_clicks_per_active_day'] = vle_features['early_sum_click'] / vle_features['early_active_days'].replace(0, np.nan)
vle_features['early_clicks_per_active_day'] = vle_features['early_clicks_per_active_day'].fillna(0)

vle_features['early_active_ratio'] = vle_features['early_active_days'] / vle_features['active_days'].replace(0, np.nan)
vle_features['early_active_ratio'] = vle_features['early_active_ratio'].fillna(0)

# activity type aggregates
vle_types = student_vle.merge(vle[['id_site','activity_type']], on='id_site', how='left')

vle_type_agg = (
    vle_types.groupby(['id_student','code_module','code_presentation','activity_type'], observed=True)
             .agg(type_clicks=('sum_click','sum'))
             .reset_index()
)

vle_type_pivot = vle_type_agg.pivot_table(
    index=['id_student','code_module','code_presentation'],
    columns='activity_type',
    values='type_clicks',
    fill_value=0
).reset_index()

for col in vle_type_pivot.columns:
    if col not in ['id_student','code_module','code_presentation']:
        vle_type_pivot = vle_type_pivot.rename(columns={col: f'clicks_type_{col}'})

vle_features = vle_features.merge(vle_type_pivot, on=['id_student','code_module','code_presentation'], how='left')

# diversity features
activity_cols = [c for c in vle_features.columns if c.startswith('clicks_type_')]
type_sum = vle_features[activity_cols].sum(axis=1).replace(0, np.nan)
type_probs = vle_features[activity_cols].div(type_sum, axis=0).fillna(0)
vle_features['distinct_activity_types'] = (vle_features[activity_cols] > 0).sum(axis=1)
vle_features['activity_entropy'] = -(type_probs * np.log(type_probs + 1e-9)).sum(axis=1)
vle_features['top_type_share'] = type_probs.max(axis=1)

vle_features['log1p_total_sum_click'] = np.log1p(vle_features['total_sum_click'])
vle_features['log1p_early_sum_click'] = np.log1p(vle_features['early_sum_click'])


C:\Users\Than Minh\AppData\Local\Temp\ipykernel_39844\1098740652.py:50: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  vle_type_pivot = vle_type_agg.pivot_table(


In [6]:
# --- assessment features (kept for completeness, later dropped)
sa = student_assess.merge(
    assessments[['id_assessment','code_module','code_presentation','assessment_type','weight','date']],
    on='id_assessment', how='left'
)

sa['weighted_score'] = np.where(sa['is_banked'] == 1, 0, sa['score'] * (sa['weight'] / 100.0))
sa['lateness_days'] = sa['date_submitted'] - sa['date']

assess_agg = (
    sa.groupby(['id_student','code_module','code_presentation'], as_index=False, observed=True)
      .agg(
          exam_weighted=('weighted_score', lambda x: x[sa.loc[x.index,'assessment_type'].eq('Exam')].sum()),
          non_exam_weighted=('weighted_score', lambda x: x[~sa.loc[x.index,'assessment_type'].eq('Exam')].sum()),
          mean_score=('score','mean'),
          late_submissions=('lateness_days', lambda s: (s>0).sum())
      )
)

assess_agg['has_exam'] = (assess_agg['exam_weighted'] > 0).astype(int)
assess_agg['has_non_exam'] = (assess_agg['non_exam_weighted'] > 0).astype(int)
assess_agg['overall_grade'] = np.where(
    (assess_agg['has_exam']==1) & (assess_agg['has_non_exam']==1),
    (assess_agg['exam_weighted'] + assess_agg['non_exam_weighted']) / 2.0,
    np.where(assess_agg['has_exam']==1, assess_agg['exam_weighted'], assess_agg['non_exam_weighted'])
)


In [7]:
# --- registration features
student_reg['date_registration'] = pd.to_numeric(student_reg['date_registration'], errors='coerce')
student_reg['date_unregistration'] = pd.to_numeric(student_reg['date_unregistration'], errors='coerce')

reg_features = student_reg[['id_student','code_module','code_presentation','date_registration','date_unregistration']].copy()
reg_features['unregistered_flag'] = reg_features['date_unregistration'].notna().astype(int)


In [8]:
# --- build final dataset
final_data = (
    student_info
      .merge(vle_features, on=['id_student','code_module','code_presentation'], how='left')
      .merge(assess_agg, on=['id_student','code_module','code_presentation'], how='left')
      .merge(reg_features, on=['id_student','code_module','code_presentation'], how='left')
      .merge(courses, on=['code_module','code_presentation'], how='left')
)

# Derived registration and engagement ratios
if 'date_registration' in final_data.columns:
    final_data['registered_early_flag'] = (final_data['date_registration'] < 0).astype(int)
    final_data['registration_lead_days'] = (-final_data['date_registration']).clip(lower=0)

if 'date_unregistration' in final_data.columns:
    final_data['unregistered_flag'] = final_data['date_unregistration'].notna().astype(int)

if 'module_presentation_length' in final_data.columns:
    final_data['active_days_ratio'] = final_data['active_days'] / final_data['module_presentation_length'].replace(0, np.nan)
    final_data['active_days_ratio'] = final_data['active_days_ratio'].fillna(0)

# Drop rows with missing values in this feature list
feature_cols = [
    'total_sum_click','active_days','early_sum_click','early_active_days','early_click_ratio',
    'clicks_per_active_day','early_clicks_per_active_day','early_active_ratio',
    'distinct_sites','distinct_activity_types','activity_entropy','top_type_share',
    'log1p_total_sum_click','log1p_early_sum_click','exam_weighted','non_exam_weighted',
    'mean_score','late_submissions','overall_grade','unregistered_flag',
    'registered_early_flag','registration_lead_days','active_days_ratio'
]

row_na_cols = [c for c in feature_cols if c in final_data.columns]
if row_na_cols:
    before = len(final_data)
    final_data = final_data.dropna(subset=row_na_cols)
    print('Dropped rows with NA in feature list:', before - len(final_data))

# remove zero-activity rows
zero_cols = [c for c in ['total_sum_click','active_days','early_sum_click','early_active_days','distinct_sites'] if c in final_data.columns]
if zero_cols:
    zero_mask = (final_data[zero_cols].sum(axis=1) == 0)
    final_data = final_data.loc[~zero_mask].copy()

# drop leakage features
LEAKAGE_FEATURES = ['overall_grade','exam_weighted','non_exam_weighted','mean_score','late_submissions']
leak_drop = [c for c in LEAKAGE_FEATURES if c in final_data.columns]
if leak_drop:
    final_data = final_data.drop(columns=leak_drop)
    print('Dropped leakage features:', leak_drop)


Dropped rows with NA in feature list: 6827
Dropped leakage features: ['overall_grade', 'exam_weighted', 'non_exam_weighted', 'mean_score', 'late_submissions']


## 4. H2O AutoML

We run AutoML to test multiple model families and return the best models ranked by the selected metric.


## 4. Baseline Models

We first train a few standard models to establish a baseline before running AutoML. This gives a clear point of comparison for later tuning.



In [9]:
# Baseline H2O models
h2o.init(max_mem_size='4G', verbose=True, log_level='INFO', log_dir='./h2o_logs')

# Prepare H2O frame
h2o_df = h2o.H2OFrame(final_data)
target = 'final_result_upd_numeric'
features = [c for c in h2o_df.columns if c != target]
h2o_df[target] = h2o_df[target].asfactor()
train, valid = h2o_df.split_frame(ratios=[0.8], seed=42)

from h2o.estimators import H2OGradientBoostingEstimator, H2ORandomForestEstimator, H2OGeneralizedLinearEstimator

baseline_models = {
    'GLM': H2OGeneralizedLinearEstimator(family='binomial'),
    'GBM': H2OGradientBoostingEstimator(),
    'DRF': H2ORandomForestEstimator()
}

baseline_rows = []
for name, model in baseline_models.items():
    model.train(x=features, y=target, training_frame=train, validation_frame=valid)
    perf = model.model_performance(valid=True)
    acc = perf.accuracy()[0][1] if perf.accuracy() else None
    f1 = perf.F1()[0][1] if perf.F1() else None
    baseline_rows.append({'model': name, 'accuracy': acc, 'f1': f1})

baseline_results = pd.DataFrame(baseline_rows).sort_values('accuracy', ascending=False)
baseline_results



print('H2O cluster status:')
h2o.cluster().show_status()



Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.12+8-LTS-286, mixed mode, sharing)
  Starting server from C:\Users\Than Minh\AppData\Roaming\Python\Python311\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\THANMI~1\AppData\Local\Temp\tmpyv5jhnv3
  JVM stdout: C:\Users\THANMI~1\AppData\Local\Temp\tmpyv5jhnv3\h2o_Than_Minh_started_from_python.out
  JVM stderr: C:\Users\THANMI~1\AppData\Local\Temp\tmpyv5jhnv3\h2o_Than_Minh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Ho_Chi_Minh
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 7 days
H2O_cluster_name:,H2O_from_python_Than_Minh_9754vv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
H2O cluster status:


H2O_cluster_uptime:,12 secs
H2O_cluster_timezone:,Asia/Ho_Chi_Minh
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 7 days
H2O_cluster_name:,H2O_from_python_Than_Minh_9754vv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.949 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [10]:
# Initialize H2O
h2o.init(max_mem_size='4G', verbose=True, log_level='INFO', log_dir='./h2o_logs')

# Prepare H2O frame
h2o_df = h2o.H2OFrame(final_data)

# Set target and features
target = 'final_result_upd_numeric'
features = [c for c in h2o_df.columns if c != target]

# Ensure target is treated as categorical
h2o_df[target] = h2o_df[target].asfactor()

# Train/valid split
train, valid = h2o_df.split_frame(ratios=[0.8], seed=42)

# AutoML search
aml = H2OAutoML(
    max_runtime_secs=1200,
    max_models=20,
    balance_classes=True,
    sort_metric='AUC',
    seed=42
)

aml.train(x=features, y=target, training_frame=train, validation_frame=valid)

lb = aml.leaderboard
lb


print('H2O cluster status:')
h2o.cluster().show_status()



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,27 secs
H2O_cluster_timezone:,Asia/Ho_Chi_Minh
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 7 days
H2O_cluster_name:,H2O_from_python_Than_Minh_9754vv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.952 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
00:02:52.351: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
00:02:52.358: AutoML: XGBoost is not available; skipping it.

█████████████████████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_8b90b862b444f03faa84f54df346a6> was cancelled by the user.

## 5. Evaluate top models

We compute Accuracy and F1 for the top models on the validation set.


In [ ]:
# Evaluate top models on validation set
leaderboard = aml.leaderboard.as_data_frame()

results = []
for model_id in leaderboard['model_id'].head(10):
    model = h2o.get_model(model_id)
    perf = model.model_performance(valid=True)
    acc = perf.accuracy()[0][1] if perf.accuracy() else None
    f1 = perf.F1()[0][1] if perf.F1() else None
    results.append({'model_id': model_id, 'accuracy': acc, 'f1': f1})

results_df = pd.DataFrame(results).sort_values('accuracy', ascending=False)
results_df
